In [4]:
# coding: utf-8

from past.utils import old_div
import numpy as np
import scipy.spatial.distance as sd
from scipy.linalg import toeplitz
import hypertools as hyp
import pandas as pd
gaussian_params = {'var': 1000}


def gaussian_weights(T, params=gaussian_params):
    c1 = np.divide(1, np.sqrt(2 * np.math.pi * params['var']))
    c2 = np.divide(-1, 2 * params['var'])
    sqdiffs = toeplitz(np.arange(T)) ** 2
    return c1 * np.exp(c2 * sqdiffs)




In [5]:
data_list= np.random.randn(10,3)
pandas_dataframe= pd.DataFrame(np.random.randint(low=0, high=10, size=(2, 2)))
numpy_array= np.array([[5, 9], [10, 7], [5, 9]])
numpy_array_list= np.array([[8,2],[4,6]]).tolist()

In [6]:
data_dl = hyp.tools.format_data(data_list)
data_pdf = hyp.tools.format_data(pandas_dataframe)
data_npa = hyp.tools.format_data(numpy_array)

In [7]:
def wisfc(data, timepoint_weights, subject_weights=None):
    if type(data) != list:
        sum = 2 * wcorr(data, data, timepoint_weights)
        sum[np.isinf(sum) | np.isnan(sum)] = 0
        S = 1
        K = data.shape[1]
        T = data.shape[0]
    elif len(data) == 1:
        sum = 2 * wcorr(data[0], data[0], timepoint_weights)
        sum[np.isinf(sum) | np.isnan(sum)] = 0
        S = 1
        K = data[0].shape[1]
        T = data[0].shape[0]
    else:
        subjects = np.arange(len(data))
        S = len(subjects)
        K = data[0].shape[1]
        T = data[0].shape[0]

        if subject_weights is None:
            connectomes = np.zeros([S, int((K**2 - K) / 2)])
            for s in subjects:
                connectomes[s, :] = 1 - sd.pdist(data[s].T, metric='correlation')
            subject_weights = 1 - sd.squareform(sd.pdist(connectomes.T,
                                                metric='correlation'))
        else:
            subject_weights = np.tile(subject_weights, [S, 1])

        sum = np.zeros([K, K, T])
        for s in subjects:
            a = data[s]
            other_inds = list([subjects[subjects != s]][0])
            b = weighted_mean(np.stack([data[x] for x in other_inds], axis=2),
                              axis=2, weights=subject_weights[s, other_inds])

            next = wcorr(a, b, timepoint_weights)
            for t in np.arange(T):
                x = next[:, :, t]
                x[np.isinf(x) | np.isnan(x)] = 0
                z = r2z(x)
                sum[:, :, t] = np.nansum(np.stack([sum[:, :, t], z + z.T],
                                                  axis=2), axis=2)

    corrs = np.zeros([T, int(((K**2 - K) / 2) + K)])
    for t in np.arange(T):
        corrs[t, :] = mat2vec(np.squeeze(z2r(np.divide(sum[:, :, t], 2*S))))

    corrs[np.isinf(corrs)] = np.sign(corrs[np.isinf(corrs)])
    corrs[np.isnan(corrs)] = 0
    return corrs

In [8]:
wisfc(data_list,)

TypeError: wisfc() missing 1 required positional argument: 'timepoint_weights'